In [37]:
# 1-Using Naïve Bayes on dataset (use 75% for training and 25% for test), 
# predict not_MDR column by two other columns. Calculate accuracy of the model on the test data.

# 2- Using all records in amr_ds.csv dataset to calculate following formula :
# amp_pen = number of records with Ampicillin=1 and Penicillin =1
# amp_nmdr = number of records with Ampicillin=1 and Not_MDR =1
# pen_ nmdr = number of records with Penicillin =1 and Not_MDR =1

# (Python Code:
# import numpy as np
# data = np.loadtxt(fname='amr_ds.csv', delimiter=',', skiprows=1)
# amp_pen = np.sum( (data[:, 0] == 1) & (data[:, 1] == 1) )
# amp_nmdr  = np.sum( (data[:, 0] == 1) & (data[:, 2] == 1) )
# pen_nmdr = np.sum( (data[:, 1] == 1) & (data[:, 2] == 1) )

# )
# Using states [‘Ampicillin’,’ Penicillin’,’ Not_MDR’]  and 
# following transition matrix develop a code to create a Markov chain.
# transition_matrix = [[0, amp_pen/(amp_nmdr+amp_pen), amp_nmdr/(amp_nmdr+amp_pen)],
#                                     [amp_pen/(pen_nmdr+amp_pen), 0, pen_nmdr/(pen_nmdr+amp_pen)],
#                                     [amp_nmdr/(amp_nmdr+pen_nmdr), pen_nmdr/(amp_nmdr+pen_nmdr), 0]]
# Calculate the probability of each state in long term (stationary state).

# image.png
# Suppose that there is an association between infection after surgery and resistances to antimicrobials as follows:

# No Infection
# Infection
# Amp
# 0.4
# 0.6
# Pen
# 0.3
# 0.7
# NMDR
# 0.8
# 0.2
# Predict most probable sequence of states if we observe the following sequence in a patient:
# [Infection after surgery, No infection after surgery, Infection after surgery]
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from scipy.linalg import eig

data = pd.read_csv('amr_ds.csv')
X = data[['Ampicillin', 'Penicillin']]
y = data['Not_MDR']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
data.head(5)

,Ampicillin,Penicillin,Not_MDR
0,1,1,0
1,0,1,1
2,0,1,1
3,0,1,1
4,0,1,0


In [38]:
# Create a BernoulliNBmodel
model = BernoulliNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of Naive Bayes model: {accuracy:.2f}')

Accuracy of Naive Bayes model: 0.95


In [39]:
# Calculate amp_pen, amp_nmdr, pen_nmdr
data_np = data.to_numpy()
amp_pen = np.sum((data_np[:, 0] == 1) & (data_np[:, 1] == 1))
amp_nmdr = np.sum((data_np[:, 0] == 1) & (data_np[:, 2] == 1))
pen_nmdr = np.sum((data_np[:, 1] == 1) & (data_np[:, 2] == 1))
print(f'amp_pen: {amp_pen}, amp_nmdr: {amp_nmdr}, pen_nmdr: {pen_nmdr}')

amp_pen: 107, amp_nmdr: 6, pen_nmdr: 55


In [40]:
# Create transition matrix
transition_matrix = np.array([
    [0, amp_pen / (amp_nmdr + amp_pen), amp_nmdr / (amp_nmdr + amp_pen)],
    [amp_pen / (pen_nmdr + amp_pen), 0, pen_nmdr / (pen_nmdr + amp_pen)],
    [amp_nmdr / (amp_nmdr + pen_nmdr), pen_nmdr / (amp_nmdr + pen_nmdr), 0]
])
# Calculate stationary state
eigenvalues, eigenvectors = eig(transition_matrix.T)
stationary_state = np.real(eigenvectors[:, np.isclose(eigenvalues, 1)])
stationary_state = stationary_state / np.sum(stationary_state)
print('Stationary state probabilities:', stationary_state.flatten())
flat_stationary_probs = stationary_state.flatten()

Stationary state probabilities: [0.33630952 0.48214286 0.18154762]


In [ ]:
# Define states and observation sequence
states = ['Ampicillin', 'Penicillin', 'Not_MDR']
observations = ['Infection', 'No Infection', 'Infection']
# Define emission probabilities
emission_probs = {
    'Ampicillin': {'No Infection': 0.4, 'Infection': 0.6},
    'Penicillin': {'No Infection': 0.3, 'Infection': 0.7},
    'Not_MDR': {'No Infection': 0.8, 'Infection': 0.2}
}
# Define transition probabilities
transition_probs = {
    'Ampicillin': {'Ampicillin': 0.0, 'Penicillin': amp_pen / (amp_nmdr + amp_pen), 'Not_MDR': amp_nmdr / (amp_nmdr + amp_pen)},
    'Penicillin': {'Ampicillin': amp_pen / (pen_nmdr + amp_pen), 'Penicillin': 0.0, 'Not_MDR': pen_nmdr / (pen_nmdr + amp_pen)},
    'Not_MDR': {'Ampicillin': amp_nmdr / (amp_nmdr + pen_nmdr), 'Penicillin': pen_nmdr / (amp_nmdr + pen_nmdr), 'Not_MDR': 0.0}
}
# Viterbi algorithm
def viterbi(observations, states, start_probs, transition_probs, emission_probs):
    V = [{}]
    path = {}
    for state in states:
        V[0][state] = start_probs[state] * emission_probs[state][observations[0]]
        path[state] = [state]
    for t in range(1, len(observations)):
        V.append({})
        new_path = {}
        for curr_state in states:
            (prob, prev_state) = max(
                (V[t-1][prev_state] * transition_probs[prev_state][curr_state] * emission_probs[curr_state][observations[t]], prev_state)
                for prev_state in states
            )
            V[t][curr_state] = prob
            new_path[curr_state] = path[prev_state] + [curr_state]
        path = new_path
    (prob, state) = max((V[len(observations) - 1][state], state) for state in states)
    return (prob, path[state])
# define start_probs with the previous stationary probabilities
start_probs = {
    states[0]: flat_stationary_probs[0],
    states[1]: flat_stationary_probs[1],
    states[2]: flat_stationary_probs[2]
}
(prob, state_sequence) = viterbi(observations, states, start_probs, transition_probs, emission_probs)
print('Most probable state sequence:', state_sequence)

Most probable state sequence: ['Penicillin', 'Ampicillin', 'Penicillin']
